# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end

aluminium_setup (generic function with 2 methods)

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trival statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298461990220                   -0.85    4.1
  2   -8.300220252516       -2.75       -1.25    1.0
  3   -8.300451143655       -3.64       -1.89    3.4
  4   -8.300462489215       -4.95       -2.76    3.2
  5   -8.300463938802       -5.84       -3.03    4.2
  6   -8.300464277376       -6.47       -3.21    1.0
  7   -8.300464455648       -6.75       -3.35   15.1
  8   -8.300464553328       -7.01       -3.48    1.1
  9   -8.300464623123       -7.16       -3.70    1.0
 10   -8.300464636831       -7.86       -3.88    1.0
 11   -8.300464643601       -8.17       -4.24    1.0


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.64186299392                   -0.70    5.5
  2   -16.67861828433       -1.43       -1.14    1.6
  3   -16.67922451086       -3.22       -1.87    3.2
  4   -16.67928229128       -4.24       -2.77    3.4
  5   -16.67928618356       -5.41       -3.13    6.2
  6   -16.67928621563       -7.49       -3.53    1.2
  7   -16.67928622278       -8.15       -4.03    1.5


In [5]:
self_consistent_field(aluminium_setup(4); is_converged, n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32526947856                   -0.56    6.6
  2   -33.33272200021       -2.13       -1.00    1.2
  3   -33.33589641182       -2.50       -1.68    3.6
  4   -33.33686532516       -3.01       -2.39    3.4
  5   -33.33693107080       -4.18       -2.89    7.5
  6   -33.33694382039       -4.89       -3.74    5.4
  7   -33.33694390910       -7.05       -4.14   12.9


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298536847036                   -0.85    4.5
  2   -8.300280031022       -2.76       -1.59    1.5
  3   -8.300460064197       -3.74       -2.66    5.9
  4   -8.300435181663   +   -4.60       -2.52   13.8
  5   -8.300464509246       -4.53       -3.69    2.4
  6   -8.300464593650       -7.07       -3.96    3.8
  7   -8.300464643769       -7.30       -5.05    3.2


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing(), n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32564510522                   -0.56    6.4
  2   -33.33208247398       -2.19       -1.28    1.0
  3   -32.66658450698   +   -0.18       -1.15    6.5
  4   -32.39662393445   +   -0.57       -1.08    5.9
  5   -33.31819037511       -0.04       -1.62    6.0
  6   -33.27750610335   +   -1.39       -1.63    3.2
  7   -33.21088989444   +   -1.18       -1.50    4.6
  8   -33.33652162468       -0.90       -2.50    3.9
  9   -33.33685118882       -3.48       -2.77    3.4
 10   -33.33674780394   +   -3.99       -2.59    3.1
 11   -33.33694093907       -3.71       -3.65    2.6
 12   -33.33694250685       -5.80       -3.81    3.5
 13   -33.33694360169       -5.96       -4.07    3.6


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.